In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

In [15]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3822150905&f_SB2=1&f_TPR=a1705895389-&f_WT=1%2C2%2C3&geoId=106981407&keywords=operations%20analyst&location=Ohio%2C%20United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R")

jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')

for listing in jobs_listing:
    job_title_element = listing.find_element(By.CLASS_NAME, 'base-search-card__title')
    job_title = job_title_element.text

    company_element = listing.find_element(By.CLASS_NAME, 'base-search-card__subtitle')
    company = company_element.text

    address_element = listing.find_element(By.CLASS_NAME, 'job-search-card__location')
    address = address_element.text

    try:
        salary_element = listing.find_element(By.CLASS_NAME, 'job-search-card__salary-info')
        salary = salary_element.text
    except:
        salary = "0"

    print("Job Title:", job_title)
    print("Company:", company)
    print("Address:", address)
    print("Salary:", salary)
    print("-" * 50)

    #job-search-card__salary-info

Job Title: Transportation/I-Trade Operations Analyst
Company: Procter & Gamble
Address: Cincinnati, OH
--------------------------------------------------
Job Title: Supply Chain Analyst
Company: Confidential
Address: Mentor, OH
--------------------------------------------------
Job Title: Operations Manager - Manufacturing
Company: CyberCoders
Address: Crooksville, OH
--------------------------------------------------
Job Title: Operations Manager - Manufacturing
Company: CyberCoders
Address: Columbus, OH
--------------------------------------------------
Job Title: SI&OP Senior Analyst
Company: The AZEK Company
Address: Wilmington, OH
--------------------------------------------------
Job Title: Sales Operations Analyst
Company: Lakeview Farms
Address: West Chester, OH
--------------------------------------------------
Job Title: SQL Reporting Analyst
Company: Watson's
Address: Cincinnati, OH
--------------------------------------------------
Job Title: Operations Manager
Company: Rel